In [1]:
import time
import  hmac
import  hashlib
import  base64

import requests
import  pandas as pd
import _private_

BASE_URL = 'https://api.naver.com'
API_KEY = _private_.my_api_key()
SECRET_KEY = _private_.my_secrete_key()
CUSTOMER_ID = _private_.my_customer_id()

def generate(timestamp, method, uri, secret_key):
    message = "{}.{}.{}".format(timestamp, method, uri)
    #hash = hmac.new(bytes(secret_key, "utf-8"), bytes(message, "utf-8"), hashlib.sha256)
    hash = hmac.new(secret_key.encode("utf-8"), message.encode("utf-8"), hashlib.sha256)
    hash.hexdigest()
    return base64.b64encode(hash.digest())

def get_header(method, uri, api_key, secret_key, customer_id):
    timestamp = str(int(time.time() * 1000))
    signature = generate(timestamp, method, uri, SECRET_KEY)
    return {'Content-Type': 'application/json; charset=UTF-8', 'X-Timestamp': timestamp, 'X-API-KEY': API_KEY, 'X-Customer': str(CUSTOMER_ID), 'X-Signature': signature}

In [2]:
def clean_result(_df_):
    df = _df_.copy()
    df.rename({
        'compIdx':'경쟁정도',
        'monthlyAveMobileClkCnt':'월평균클릭수_모바일',
        'monthlyAveMobileCtr':'월평균클릭률_모바일',
        'monthlyAvePcClkCnt':'월평균클릭수_PC',
        'monthlyAvePcCtr':'월평균클릭률_PC', 
        'monthlyMobileQcCnt':'월간검색수_모바일',
        'monthlyPcQcCnt': '월간검색수_PC',
        'plAvgDepth':'월평균노출광고수', 
        'relKeyword':'연관키워드'
        }, axis=1,inplace=True)
    df.head()
    df.set_index('연관키워드',inplace=True)
    
    _dict_ = {'높음':3,'중간':2,'낮음':1}
    df['경쟁정도']=df['경쟁정도'].map(_dict_)
    for col in df.columns:
        df[col] = pd.to_numeric(df[col],errors='coerce')
    df = df.dropna()
    return df

In [3]:
import pickle
with open('raw_data/daily_trend/2022_11_12.pickle','rb') as f:
    _dict_=pickle.load(f)

In [4]:
pd.DataFrame(_dict_)

,패션의류,패션잡화,화장품/미용,디지털/가전,가구/인테리어,출산/육아,식품,스포츠/레저,생활/건강,여가/생활편의,면세점,도서
0,잠옷,실내화,샴푸,가습기,암막커튼,물티슈,오메가3,무릎보호대,덴탈마스크,화환,면세점가방,학지사
1,내셔널지오그래픽패딩,양말,트리트먼트,탄소매트,선반,답례품,양배추즙,핫팩,kf94마스크새부리형,개업화분,면세점향수,나의문화유산답사기
2,맨투맨,털슬리퍼,바디워시,로봇청소기,러그,젖병소독기,비오틴,허리보호대,차량용핸드폰거치대,근조화환,n224mdw020,ashgatepubco
3,남자니트,페이크삭스,탈모샴푸,가열식가습기,행거,순산양분유,절임배추,캠핑랜턴,차량용방향제,꽃바구니,n224wdw020,알에이치코리아
4,남자팬티,어그슬리퍼,바디로션,무선청소기,디퓨저,어린이마스크,락토페린,손목보호대,크리스마스트리,개업화환,titoni,집문당
5,원피스,크록스,마스크팩,프린터,베개,백일상대여,고구마,골프거리측정기,싱크대수전,꽃배달,면세점만다리나덕,법문사
6,남성팬티,나이키운동화,클렌징폼,온풍기,매트리스,층간소음매트,과메기,캠핑의자,전자담배액상,부산요트투어,cpb,경문사
7,여성코트,운동화,수분크림,라디에이터,이불,유아마스크,밀크씨슬,침낭,안마의자,문화상품권,다미아니귀걸이,세광음악출판사
8,후드티,14k목걸이,선크림,tv,모션데스크,포켓몬마스크,비타민d,캠핑테이블,철제선반,전국꽃배달,만다리나덕면세점,동양북스
9,커플잠옷,깔창,쿠션,전기히터,매트리스커버,아기이불,사과,발열조끼,kf94마스크,제주스냅,수려한골든콜라겐쿠션,베스트셀러순위


In [5]:
_dict_mapper={'패션의류':'fashion_clothes',
 '패션잡화':'fashion_etc',
 '화장품/미용':'cosmetic',
 '디지털/가전':'electronics',
 '가구/인테리어':'furnitures',
 '출산/육아':'baby',
 '식품':'food',
 '스포츠/레저':'sports',
 '생활/건강':'life_health',
 '여가/생활편의':'convinence',
 '면세점':'tax_free',
 '도서':'book'}

In [6]:
tot_df = list()
for key in _dict_.keys():
    temp_tot = list()
    for title in _dict_[key]:
        keyword = title #키워드검색
        dic_return_kwd = {}
        naver_ad_url = '/keywordstool'
        
        #_kwds_string = ['나이키', '원피스', '운동화'] #키워드 여러개일경우
        method = 'GET'
        prm = {'hintKeywords' : keyword , 'showDetail':1}

        r = requests.get(BASE_URL + naver_ad_url, params=prm, headers=get_header(method, naver_ad_url, API_KEY, SECRET_KEY, CUSTOMER_ID))

        r_data = r.json()
        naver_ad_summary = pd.DataFrame(r_data['keywordList'])  

        df_raw=naver_ad_summary.copy()
        df = df_raw.copy()
        clean_df=clean_result(df)
        temp_tot.append(clean_df)
        time.sleep(.5)
    time.sleep(.5)
    vars()[f'df_{_dict_mapper[key]}'] = pd.concat(temp_tot)
    tot_df.append(vars()[f'df_{_dict_mapper[key]}'])

In [8]:
df_tot=pd.concat(tot_df)
df_tot.drop_duplicates(inplace=True)
df_tot[(df_tot['월간검색수_PC']>1000) & (df_tot['월간검색수_모바일']>1000)].reset_index().to_csv('2022_11_14_main.csv')

In [ ]:
_dict_mapper={'패션의류':'fashion_clothes',
 '패션잡화':'fashion_etc',
 '화장품/미용':'cosmetic',
 '디지털/가전':'electronics',
 '가구/인테리어':'furnitures',
 '출산/육아':'baby',
 '식품':'food',
 '스포츠/레저':'sports',
 '생활/건강':'life_health',
 '여가/생활편의':'convinence',
 '면세점':'tax_free',
 '도서':'book'}